In [1]:
#Instalación de Librerias
import pandas as pd
import numpy as np
import math
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr
import scipy.stats as stats
from scipy import stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest, t, levene, bartlett, chi2_contingency
from scipy.stats import ks_2samp
from scipy.stats import bartlett
# -----------------------------------------------------------------------

# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) 
import warnings

In [2]:
# LLamo la función para leer csv e imprimir las 5 primeras filas 
df_Study_Data = pd.read_csv('Case_Study_Data_2024.csv', encoding='latin1', index_col=0)
df_Study_Data.sample(2)

,brand,box_office_revenue,opening_revenue,release_date,opening_revenue_over_total_revenue,imdb_score,rt_critics_score,rt_audience_score
film_title,,,,,,,,
Finding Dory,Pixar,"$486,295,561","$135,060,273",6/17/2016,28,7.2,94,84
The Princess and the Frog,Disney Animated Feature,"$104,400,899","$786,190",11/25/2009,1,7.1,85,74


In [3]:
# Utilizamos una función básica de exploración que hemos creado previamente

def exploracion_dataframe (dataframe):
    """
    Realiza un análisis exploratorio básico de un DataFrame, mostrando información sobre duplicados,
    valores nulos, tipos de datos, valores únicos para columnas categóricas y estadísticas descriptivas
    para columnas categóricas y numéricas, agrupadas por la columna de control.

    Parámetros:
    - dataframe (DataFrame): El DataFrame que se va a explorar.

    Returns: 
    No devuelve nada directamente, pero imprime en la consola la información exploratoria.
    """
    
    print(f"Número de filas: {dataframe.shape[0]}. Número de columnas: {dataframe.shape[1]}")
    print("\n ..................... \n")

    print(f"Número de duplicados: {dataframe.duplicated().sum()}")
    print("\n ..................... \n")
    
    print("Porcentaje de nulos:")
    df_nulos = pd.DataFrame(dataframe.isnull().sum() / dataframe.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])
    
    print("\n ..................... \n")
    print(f"Tipos de columnas:")
    display(pd.DataFrame(dataframe.dtypes, columns = ["tipo_dato"]))
    
    print("\n ..................... \n")
    print("Valores de las columnas categóricas: ")
    dataframe_categoricas = dataframe.select_dtypes(include = "O")
    
    for col in dataframe_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valores únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()))    
    
    print("\n ..................... \n")
    print("Valores de las columnas numéricas: ")
    dataframe_numericas = dataframe.select_dtypes(include='number')
    
    for col in dataframe_numericas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valores únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()))    

    print("\n ..................... \n")
    descripcion = dataframe.describe().T
    print(f"Principales valores estadísticos son:")
    print(descripcion.to_string())


In [4]:
exploracion_dataframe(df_Study_Data)

Número de filas: 119. Número de columnas: 8

 ..................... 

Número de duplicados: 0

 ..................... 

Porcentaje de nulos:


,%_nulos



 ..................... 

Tipos de columnas:


,tipo_dato
brand,object
box_office_revenue,object
opening_revenue,object
release_date,object
opening_revenue_over_total_revenue,int64
imdb_score,float64
rt_critics_score,int64
rt_audience_score,int64



 ..................... 

Valores de las columnas categóricas: 
La columna BRAND tiene las siguientes valores únicos:


...

La columna RT_CRITICS_SCORE tiene las siguientes valores únicos:


,count
rt_critics_score,
90,6
94,6
91,6
93,5
88,4
79,4
98,4
89,4
84,4


La columna RT_AUDIENCE_SCORE tiene las siguientes valores únicos:


,count
rt_audience_score,
92,8
87,7
94,6
75,6
85,5
90,5
86,5
84,4
70,3



 ..................... 

Principales valores estadísticos son:
                                    count       mean        std   min    25%   50%   75%    max
opening_revenue_over_total_revenue  119.0  27.882353  12.370123   0.0  24.00  29.0  35.0   64.0
imdb_score                          119.0   7.119328   0.759826   4.8   6.75   7.2   7.7    8.5
rt_critics_score                    119.0  76.655462  19.545877  21.0  68.00  84.0  91.0  100.0
rt_audience_score                   119.0  77.789916  16.171682  29.0  71.50  83.0  90.0   98.0


In [5]:
def remove_dollar_sign(value):
    return float(value.replace('$', '').replace(',', '').strip())

# Aplicar la función a las columnas seleccionadas
columns_to_clean = ['box_office_revenue', 'opening_revenue']
for col in columns_to_clean:
    df_Study_Data[col] = df_Study_Data[col].apply(remove_dollar_sign)

# Mostrar el DataFrame
print(df_Study_Data)

                                                               brand  \
film_title                                                             
Onward                                                         Pixar   
The Marvels                                                   Marvel   
Ant-Man and the Wasp: Quantumania                             Marvel   
Doctor Strange in the Multiverse of Madness                   Marvel   
Captain America: Civil War                                    Marvel   
...                                                              ...   
A Bug's Life                                                   Pixar   
Toy Story 2                                                    Pixar   
Beauty and the Beast                         Disney Animated Feature   
Aladdin                                      Disney Animated Feature   
Frozen                                       Disney Animated Feature   

                                             box_office_revenue

In [6]:
df_Study_Data.head()

,brand,box_office_revenue,opening_revenue,release_date,opening_revenue_over_total_revenue,imdb_score,rt_critics_score,rt_audience_score
film_title,,,,,,,,
Onward,Pixar,61555145.0,39119861.0,3/6/2020,64,7.4,88,95
The Marvels,Marvel,84500223.0,46110859.0,11/10/2023,55,5.6,62,82
Ant-Man and the Wasp: Quantumania,Marvel,214504909.0,106109650.0,2/17/2023,50,6.5,47,82
Doctor Strange in the Multiverse of Madness,Marvel,411331607.0,187420998.0,5/6/2022,46,7.0,74,85
Captain America: Civil War,Marvel,408084349.0,179139142.0,5/6/2016,44,7.8,90,89


In [7]:
df_Study_Data.to_csv('Study_Data_Clean.csv')